# predicting house prices 
## house sales in King county, USA.
### libraries used: Fast.Ai and Scikit learn

### note: this is the first kernel i've made with zero visualization. meant to be taken as a personal documentation

In [164]:
#imports
from fastai.imports import *
from fastai.structured import *

from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display
from pandas_summary import DataFrameSummary



In [165]:
data_path = "/Users/abilash/Desktop/kc_house_data.csv"
#let's call dataframe as df.
df_raw = pd.read_csv(data_path, low_memory=False, parse_dates=["date"])
display(df_raw)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,2014-10-13,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,2014-12-09,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,2015-02-25,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,2014-12-09,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,2015-02-18,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503
5,7237550310,2014-05-12,1225000.0,4,4.50,5420,101930,1.0,0,0,...,11,3890,1530,2001,0,98053,47.6561,-122.005,4760,101930
6,1321400060,2014-06-27,257500.0,3,2.25,1715,6819,2.0,0,0,...,7,1715,0,1995,0,98003,47.3097,-122.327,2238,6819
7,2008000270,2015-01-15,291850.0,3,1.50,1060,9711,1.0,0,0,...,7,1060,0,1963,0,98198,47.4095,-122.315,1650,9711
8,2414600126,2015-04-15,229500.0,3,1.00,1780,7470,1.0,0,0,...,7,1050,730,1960,0,98146,47.5123,-122.337,1780,8113
9,3793500160,2015-03-12,323000.0,3,2.50,1890,6560,2.0,0,0,...,7,1890,0,2003,0,98038,47.3684,-122.031,2390,7570


In [166]:
#df_raw.date is a column full of date and time stamps.
#assigning the fastAi method to split them into separate date month year time ect]
df_raw.price = np.log(df_raw.price)


df_raw["Date"] = pd.to_datetime(df_raw['date']).dt.day
df_raw["Month"] = pd.to_datetime(df_raw['date']).dt.month
df_raw["Year"] = pd.to_datetime(df_raw['date']).dt.year
df_raw["Hour"] = pd.to_datetime(df_raw['date']).dt.hour
df_raw["Minute"] = pd.to_datetime(df_raw['date']).dt.minute




df_raw = df_raw.drop('date', axis = 1)



In [167]:
display(df_raw.T)

,0,1,2,3,4,5,6,7,8,9,...,21603,21604,21605,21606,21607,21608,21609,21610,21611,21612
id,7.129301e+09,6.414100e+09,5.631500e+09,2.487201e+09,1.954401e+09,7.237550e+09,1.321400e+09,2.008000e+09,2.414600e+09,3.793500e+09,...,7.852140e+09,9.834201e+09,3.448900e+09,7.936000e+09,2.997800e+09,2.630000e+08,6.600060e+09,1.523300e+09,2.913101e+08,1.523300e+09
price,1.230998e+01,1.319561e+01,1.210071e+01,1.331133e+01,1.314217e+01,1.401845e+01,1.245877e+01,1.258400e+01,1.234366e+01,1.268541e+01,...,1.313676e+01,1.296921e+01,1.332234e+01,1.382298e+01,1.307107e+01,1.279386e+01,1.289922e+01,1.290446e+01,1.289922e+01,1.269158e+01
bedrooms,3.000000e+00,3.000000e+00,2.000000e+00,4.000000e+00,3.000000e+00,4.000000e+00,3.000000e+00,3.000000e+00,3.000000e+00,3.000000e+00,...,3.000000e+00,3.000000e+00,4.000000e+00,4.000000e+00,3.000000e+00,3.000000e+00,4.000000e+00,2.000000e+00,3.000000e+00,2.000000e+00
bathrooms,1.000000e+00,2.250000e+00,1.000000e+00,3.000000e+00,2.000000e+00,4.500000e+00,2.250000e+00,1.500000e+00,1.000000e+00,2.500000e+00,...,2.500000e+00,2.000000e+00,2.500000e+00,3.500000e+00,2.500000e+00,2.500000e+00,2.500000e+00,7.500000e-01,2.500000e+00,7.500000e-01
sqft_living,1.180000e+03,2.570000e+03,7.700000e+02,1.960000e+03,1.680000e+03,5.420000e+03,1.715000e+03,1.060000e+03,1.780000e+03,1.890000e+03,...,2.270000e+03,1.490000e+03,2.520000e+03,3.510000e+03,1.310000e+03,1.530000e+03,2.310000e+03,1.020000e+03,1.600000e+03,1.020000e+03
sqft_lot,5.650000e+03,7.242000e+03,1.000000e+04,5.000000e+03,8.080000e+03,1.019300e+05,6.819000e+03,9.711000e+03,7.470000e+03,6.560000e+03,...,5.536000e+03,1.126000e+03,6.023000e+03,7.200000e+03,1.294000e+03,1.131000e+03,5.813000e+03,1.350000e+03,2.388000e+03,1.076000e+03
floors,1.000000e+00,2.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,2.000000e+00,1.000000e+00,1.000000e+00,2.000000e+00,...,2.000000e+00,3.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,3.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00
waterfront,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
view,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
condition,3.000000e+00,3.000000e+00,3.000000e+00,5.000000e+00,3.000000e+00,3.000000e+00,3.000000e+00,3.000000e+00,3.000000e+00,3.000000e+00,...,3.000000e+00,3.000000e+00,3.000000e+00,3.000000e+00,3.000000e+00,3.000000e+00,3.000000e+00,3.000000e+00,3.000000e+00,3.000000e+00


In [168]:
df, y, nas = proc_df(df_raw, 'price')


In [169]:
m = RandomForestRegressor(n_jobs=-1)
m.fit(df, y)
m.score(df,y)

/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.9780754055926627

In [170]:
#but obviously 🙄 simply scoring the dataset just training is not cool. Henceforth...

In [171]:
def split_vals(a,n): return a[:n].copy(), a[n:].copy()

n_valid = 12000
n_trn = len(df_raw)-n_valid
raw_train, raw_valid = split_vals(df_raw, n_trn)
x_train, x_valid = split_vals(df, n_trn)
y_train, y_valid = split_vals(y, n_trn)

x_train.shape, x_valid.shape, y_train.shape, y_valid.shape



((9613, 24), (12000, 24), (9613,), (12000,))

In [178]:
def rmse(x,y): return math.sqrt(((x-y)**2).mean())



In [181]:
#allocate log of the price variable therefore it workss better with RMSLE problem
#this dataset does not contain string value or N/A value. very noice. therefore let's create a randomforesstregressor
model = RandomForestRegressor(n_jobs = -1)
model.fit(x_train, y_train)
#model.score(x_train, y_train)
print(model.score(x_valid, y_valid), rmse(model.predict(x_valid), y_valid) )

/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.869897414770995 0.1892711973156719
